In [26]:
!pip install --upgrade --force-reinstall git+https://github.com/hpharmsen/pysimplicate

  Cloning https://github.com/hpharmsen/pysimplicate to /private/var/folders/hc/hnz1l1_x58s508qpjcgpkp7c0000gn/T/pip-req-build-94kcfe87
  Running command git clone -q https://github.com/hpharmsen/pysimplicate /private/var/folders/hc/hnz1l1_x58s508qpjcgpkp7c0000gn/T/pip-req-build-94kcfe87
  Using cached requests-2.25.1-py2.py3-none-any.whl (61 kB)
  Using cached beautiful_date-2.0.0-py2.py3-none-any.whl (10 kB)
  Using cached six-1.15.0-py2.py3-none-any.whl (10 kB)
  Using cached python_dateutil-2.8.1-py2.py3-none-any.whl (227 kB)
  Using cached chardet-4.0.0-py2.py3-none-any.whl (178 kB)
  Using cached urllib3-1.26.3-py2.py3-none-any.whl (137 kB)
  Using cached idna-2.10-py2.py3-none-any.whl (58 kB)
  Using cached certifi-2020.12.5-py2.py3-none-any.whl (147 kB)
Using legacy 'setup.py install' for pysimplicate, since package 'wheel' is not installed.
  Attempting uninstall: six
    Found existing installation: six 1.15.0
    Uninstalling six-1.15.0:
      Successfully uninstalled six-1.1

## Uren submitten

In [27]:
from configparser import ConfigParser
import json, requests, datetime
from pysimplicate import Simplicate

ini = ConfigParser()
ini.read('../credentials.ini')

subdomain = ini['simplicate']['subdomain']
api_key = ini['simplicate']['api_key']
api_secret = ini['simplicate']['api_secret']

sim = Simplicate(subdomain, api_key, api_secret )

In [39]:
def post(url_path: str, post_fields: dict):
    headers = {
        'Authentication-Key': api_key,
        'Authentication-Secret': api_secret,
        'Content-type': 'application/json',
        'Accept': 'text/plain',
    }
    url = f'https://{subdomain}.simplicate.nl/api/v2{url_path}'
    print( 'POST', url, post_fields, headers)
    return requests.post(url, json=post_fields, headers=headers)


def get(url_path: str):
    url = f'https://{subdomain}.simplicate.nl/api/v2{url_path}'
    headers = {'Authentication-Key': api_key, 'Authentication-Secret': api_secret}
    response = requests.get(url, headers=headers, timeout=15)
    return response.json()['data']

In [29]:
employee_id = get( '/hrm/employee?q[name]=Hans-Peter Harmsen')[0]['id']
project_id = get('/projects/project?q[project_number]=OBE-1')[0]['id']
service_id = get(f'/projects/service?q[project_id]={project_id}&q[name]=Finance')[0]['id']
hourstype_id = get('/hours/hourstype?q[label]=Internal')[0]['id']

In [30]:
employee = sim.employee({'name': 'Hans-Peter Harmsen'})
employee

AttributeError: 'tuple' object has no attribute 'keys'

In [41]:
def find_bookable( sim, zoek):
    zoek = zoek.lower()
    projects = sim.project({'active':True})
    projects = {p['id']:p['organization']['name']+ ' ' + p['name'] for p in projects}
    services = sim.service({'status':'open'})

    res = []
    for s in services:
        if not s['project_id'] in projects.keys() or not s.get('name'):
            continue
        for h in s.get('hour_types',[]):
            full_name = projects[s['project_id']] + ' ' + s['name'] + ' ' + h['hourstype']['label']
            full_name = full_name.replace( ' Internal ', ' ')
            if not full_name.lower().count(zoek):
                continue
            print( s['id'] + ' ' + full_name)
            res += [( s['project_id'], s['id'], h['hourstype']['id'] )]
    if len(res) == 1:
        return res[0]
    if len(res) == 0:
        print( 'No service found')
    else:
        print( 'Multiple services found')
    return None, None, None


In [42]:
find_bookable( sim, 'on Finance')

service:1da27d3126430141d8107a68a60f150e Oberon Finance Internal


('project:21ea648f0c0c31fcfeaad60b7a7437df',
 'service:1da27d3126430141d8107a68a60f150e',
 'hourstype:3e9d74f89ca7e8deca9dc23db773baf0')

In [ ]:
find_bookable( sim, 'on Fidfnance')

In [ ]:
find_bookable( sim, 'Finance')

In [43]:
def book( search, amount, note='', date=None ):
    if not date: 
        date = datetime.datetime.now().strftime( '%Y-%m-%d')
    project_id, service_id, hourstype_id = find_bookable( sim, search)
    if search[0]:
        postdata = {
          "employee_id": employee_id,
          "project_id": project_id,
          "projectservice_id": service_id,
          "type_id": hourstype_id,
          "amount": amount,
          "start_date": date,
          "note": note
        }
        print( 'posting')
        return post( '/hours/hours', post_fields = postdata)
        #return sim.book_hours( postdata )
    else:
        print( 'nope')

In [44]:
print( book( 'on Fin', 1, 'supertest'))

service:1da27d3126430141d8107a68a60f150e Oberon Finance Internal
posting
POST https://oberon.simplicate.nl/api/v2/hours/hours {'employee_id': 'employee:41ac106f9350a0146d44e34a3f0f8c3d', 'project_id': 'project:21ea648f0c0c31fcfeaad60b7a7437df', 'projectservice_id': 'service:1da27d3126430141d8107a68a60f150e', 'type_id': 'hourstype:3e9d74f89ca7e8deca9dc23db773baf0', 'amount': 1, 'start_date': '2021-02-04', 'note': 'supertest'} {'Authentication-Key': '3KqWKCtQ6Em9HDpxsHtvcecaFv4L0nSU', 'Authentication-Secret': 'Y9GyBZexurx1VMpdkJymNBnjXhUOvCyj', 'Content-type': 'application/json', 'Accept': 'text/plain'}
<Response [200]>


In [54]:
projects = {p['id']:p['name'] for p in sim.project({'active':True})}
project_ids = projects.keys()
projects

{'project:21ea648f0c0c31fcfeaad60b7a7437df': 'Internal',
 'project:5fdd3f799c981c6dfeaad60b7a7437df': 'TOR 3.0',
 'project:b702bba66765ebadfeaad60b7a7437df': 'Homestudios Traject 2021',
 'project:2e7d1588e87a48a8feaad60b7a7437df': 'Website',
 'project:5b9aa481b594947efeaad60b7a7437df': 'Van Esch - SLA',
 'project:4537cbc119015126feaad60b7a7437df': 'Corporate website - SLA',
 'project:8a11f2a1ccca7569feaad60b7a7437df': 'Value Jagers Agile',
 'project:fc5dc3ffdea030554c13c77ab857ae53': 'JobOn Strippenkaart',
 'project:1675e5dd3dbf6d78feaad60b7a7437df': 'Slim Beleggen Agile ',
 'project:921a9486c1cbd8e6feaad60b7a7437df': 'BEN App',
 'project:57d6ce9ecef83f93feaad60b7a7437df': 'Salesforce afronden',
 'project:dac6961c7d699864feaad60b7a7437df': 'Service Team',
 'project:9b63e1bed41e11eefeaad60b7a7437df': 'Redesign website',
 'project:458f11a60675f774feaad60b7a7437df': 'Topa - SLA',
 'project:ba493253e99f604dfeaad60b7a7437df': 'Headless 2021',
 'project:ba93213f5de2e6a5feaad60b7a7437df': 'VV

In [60]:
services = [projects[s['project_id']]['name'] + ' ' + s['name'] for s in sim.service({'status':'open'}) if s['project_id'] in project_ids]
len(services)

TypeError: string indices must be integers

In [59]:
for s in sim.service({'status':'open'}):
    if s['project_id'] in project_ids:
        projects[s['project_id']]['name'] + ' ' + print(  s['name'] )

TypeError: string indices must be integers